In [220]:
pip install ultraimport

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## supress print

In [221]:
import builtins

# Step 1: Store the original print function
original_print = builtins.print

def suppress_print():
    """Temporarily suppress print statements."""
    builtins.print = lambda *args, **kwargs: None  # Override print with a no-op

def restore_print():
    """Restore the original print function."""
    builtins.print = original_print  # Restore to the saved original print function



In [222]:
import ultraimport

In [223]:
import os
os.getcwd()

'c:\\Users\\pande\\OneDrive\\Documents\\code\\DL\\DL self mini projects\\DL02-RL-tictactoe\\notebooks'

In [224]:
ultraimport('../scripts/helper.py','*', add_to_ns=globals())

{'N': 3,
 'board': array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'check_winner': <function helper.check_winner(board)>,
 'find_winning_move': <function helper.find_winning_move(board, available_moves, player_number)>,
 'get_available_moves': <function helper.get_available_moves(board)>,
 'i': 2,
 'initialize_board': <function helper.initialize_board()>,
 'is_board_full': <function helper.is_board_full(board)>,
 'isvacant': <function helper.isvacant(board, position)>,
 'j': 2,
 'make_move': <function helper.make_move(board, position, player)>,
 'np': <module 'numpy' from 'c:\\Users\\pande\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>,
 'play_game': <function helper.play_game(agent1=None, agent2=None)>,
 'print_board': <function helper.print_board(board)>,
 'random': <module 'random' from 'c:\\Users\\pande\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\random.py'>,
 'random_move': <function helper.random_move(board, available_moves)>,
 'reset_board': <

In [225]:
board = initialize_board()
board

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

## Move selection

In [226]:
# def minimax_score(board):
#     result = check_winner(board)
#     if result == 2 :
#         return -10
#     elif result == 1:
#         return 10
#     elif result == -1:
#         raise Exception("Game running!")
#     return 0

# def minimax(board, depth, alpha, beta, player_number):
#     if depth == 0:
#         return None, minimax_score(board)
#     available_moves = get_available_moves(board)
#     assert(len(available_moves) == depth)
    
#     best_move = -2
#     if player_number == 1:
#         best_score = -99
#         for move in available_moves:
#             board[move] = player_number
#             _, score = minimax(board, depth - 1, alpha, beta, 3 - player_number)
#             board[move] = 0
#             if score > best_score : 
#                 best_score = score
#                 best_move = move
#             alpha  = max(alpha, score)
#             if beta <= alpha:
#                 break
        
#     else:
#         best_score = 99
#         for move in available_moves:
#             board[move] = player_number
#             _, score = minimax(board, depth - 1, alpha, beta, 3 - player_number)
#             board[move] = 0
#             if score < best_score : 
#                 best_score = score
#                 best_move = move
#             beta =  min(beta, score)
#             if beta <= alpha:
#                 break
#     return best_move, best_score

In [227]:
import sys

def minimax_score(board):
    result = check_winner(board)
    if result == 1:  # Player 1 wins
        return 10
    elif result == 2:  # Player 2 wins
        return -10
    elif result == -1:  # Game is ongoing
        return 0  # You may also choose a neutral score like 0 for ongoing games.
    return 0  # Neutral score for draw or ongoing

def minimax(board, depth, alpha, beta, player_number):
    # Check for terminal state first
    score = minimax_score(board)
    if score != 0 or depth == 0:  # If terminal state or depth is 0
        return None, score  # Return score for evaluation

    available_moves = get_available_moves(board)

    best_move = None
    if player_number == 1:
        best_score = -sys.maxsize - 1  # Initialize to negative infinity
        for move in available_moves:
            board[move] = player_number
            _, score = minimax(board, depth - 1, alpha, beta, 2)  # Switch to player 2
            board[move] = 0  # Undo the move
            if score > best_score:
                best_score = score
                best_move = move
            alpha = max(alpha, score)
            if beta <= alpha:  # Alpha-beta pruning
                break
    else:  # Minimizing for player 2
        best_score = sys.maxsize  # Initialize to positive infinity
        for move in available_moves:
            board[move] = player_number
            _, score = minimax(board, depth - 1, alpha, beta, 1)  # Switch to player 1
            board[move] = 0  # Undo the move
            if score < best_score:
                best_score = score
                best_move = move
            beta = min(beta, score)
            if beta <= alpha:  # Alpha-beta pruning
                break

    return best_move, best_score


In [228]:
class Agent:
    def __init__(self, player_number, strategy = 'random'):
        self.player_number = player_number
        self.strategy = strategy

    def select_move(self, board, available_moves = None):
        if available_moves is None:
            available_moves = get_available_moves(board)
        if self.strategy == 'random':
            return random_move(board, available_moves)
        elif self.strategy == 'greedy':
            return self.greedy_move(board, available_moves, self.player_number)
        elif self.strategy == 'block':
            return self.greedy_move(board, available_moves, self.player_number)
        elif self.strategy == 'center_corner':
            return self.center_corner_move(board, available_moves)
        elif self.strategy == 'greedy_center':
            return self.greedy_center_move(board, available_moves, self.player_number)
        elif self.strategy == 'minmax':
            move,_ =  minimax(board, len(available_moves),-99, 99, self.player_number)
            return move
        else:
            raise Exception(f'Invalid strategy {strategy}')
        # Add more strategies as needed    
    
    def block_move(self, board, available_moves, player_number):
        #try to block opponent win
        other_player = 3 - player_number
        move = find_winning_move(board, available_moves, other_player)
        if move !=-1:
            return move
        else:
            return random_move(board, available_moves)

    def greedy_move(self, board, available_moves, player_number):
        # Try to win in single turn
        move = find_winning_move(board, available_moves, player_number)
        if move !=-1:
            return move

        # return self.block_move(board, available_moves, player_number)
        return random_move(board, available_moves)

    def center_corner_move(self, board, available_moves):
        # Flattened board indices for center, corners, and edges
        center = [(N**2 - 1) // 2]  # For a 3x3 board, center is index 4 (0-based)
        corners = [0, N-1, N**2-N, N**2-1]  # Top-left, top-right, bottom-left, bottom-right
        # edges = [i for i in range(N**2) if i not in center + corners]  # All non-corner, non-center positions
        
        # First, check if the center is available
        if board[center[0]] == 0:  # Assuming 0 means empty
            return center[0]
        
        # If center is not available, check for available corners
        available_corners = [i for i in corners if board[i] == 0]
        if available_corners:
            return random_move(board, available_corners)  # Randomly select one if multiple are available
        
        # # If neither center nor corners are available, move to an edge
        # available_edges = [i for i in edges if board[i] == 0]
        # if available_edges:
        #     return random.choice(available_edges)  # Randomly select one if multiple are available
        
        # If no moves are available (shouldn't happen if the game is valid), return None
        return random_move(board, available_moves)

    def greedy_center_move(self, board, available_moves, player_number):
        # Try to win in single turn
        move = find_winning_move(board, available_moves, player_number)
        if move !=-1:
            return move
        
        other_player = 3 - player_number
        move = find_winning_move(board, available_moves, other_player)
        if move !=-1:
            return move

        return self.center_corner_move(board, available_moves)

    
    

        


In [229]:
import random

# Define the board size
N = 3  # For a standard Tic-Tac-Toe board, N = 3

def select_center_corner_edge_move(board):
    # Flattened board indices for center, corners, and edges
    center = [(N**2 - 1) // 2]  # For a 3x3 board, center is index 4 (0-based)
    corners = [0, N-1, N**2-N, N**2-1]  # Top-left, top-right, bottom-left, bottom-right
    edges = [i for i in range(N**2) if i not in center + corners]  # All non-corner, non-center positions
    
    # First, check if the center is available
    if board[center[0]] == 0:  # Assuming 0 means empty
        return center[0]
    
    # If center is not available, check for available corners
    available_corners = [i for i in corners if board[i] == 0]
    if available_corners:
        return random.choice(available_corners)  # Randomly select one if multiple are available
    
    # If neither center nor corners are available, move to an edge
    available_edges = [i for i in edges if board[i] == 0]
    if available_edges:
        return random.choice(available_edges)  # Randomly select one if multiple are available
    
    # If no moves are available (shouldn't happen if the game is valid), return None
    return None

# Example usage
# 0 represents an empty position, 1 for player 1, and 2 for player 2
board = [1, 0, 0, 2, 0, 2, 0, 1, 1]  # Example current board state
move = select_center_corner_edge_move(board)
print(f"Selected move: {move}")


Selected move: 4


## Round robin Tournament

In [230]:
from tqdm import tqdm

In [231]:
def round_robin_tournament(strategies, rounds = 1, verbose = True, self_games = True):
    # Step 1: Create a list to hold results
    results = {strategy: {"wins": 0, "losses": 0, "draws": 0} for strategy in strategies}

    num_strategies = len(strategies)
    win_grid = np.zeros((num_strategies, num_strategies))
    strategy_indices = {strategy: idx for idx, strategy in enumerate(strategies)}  # Mapping strategy to grid index

    # Step 2: Create agents for each strategy
    agents_1 = {strategy: Agent(player_number=1, strategy=strategy) for strategy in strategies}
    agents_2 = {strategy: Agent(player_number=2, strategy=strategy) for strategy in strategies}


    # Step 3: Play games among agents
    total_matches = len(strategies) * (len(strategies) - 1 + int(self_games)) * rounds

    k = 1
    if verbose:
        suppress_print()
    with tqdm(total=total_matches, desc='Playing matches') as pbar:
        for round in range(rounds):
            
            for strategy1 in strategies:
                print()
                for strategy2 in strategies:
                    if not(self_games) and strategy1 == strategy2:  # Skip if both strategies are the same
                        
                        continue
                    

                    
                    agent1 = agents_1[strategy1]
                    agent2 = agents_2[strategy2]
                    # Step 4: Create two agents for the current match
                    
                   
                    # for match in range(2):
                    if True:
                        
                        # # Temporarily set player numbers for the match
                        # agent1.player_number = match
                        # agent2.player_number = 3 - match
                        random.seed(42)  # Initialize before the match
                        # if match == 1:
                        #     # swap(strategy1, strategy2)
                        #     strategy1, strategy2 =  strategy2, strategy1
                        

                        suppress_print()
                        # Play game and get the result
                        winner = play_game(agent1=agent1, agent2=agent2)
                        
                        
                        if verbose:
                            restore_print()
                        # Step 5: Update results based on the winner
                        if winner == 0:  # Draw
                            results[strategy1]["draws"] += 1
                            results[strategy2]["draws"] += 1
                        elif winner == 1:  # agent1 wins
                            results[strategy1]["wins"] += 1
                            results[strategy2]["losses"] += 1
                        elif winner == 2:  # agent2 wins
                            results[strategy2]["wins"] += 1
                            results[strategy1]["losses"] += 1
                        else:
                            print(winner ,"wom")


                        # Update win grid
                        idx1 = strategy_indices[strategy1]
                        idx2 = strategy_indices[strategy2]

                        if winner == 1:
                            win_grid[idx1, idx2] += 1  # Increment win count for strategy1 against strategy2
                        elif winner == 2:
                            win_grid[idx2, idx1] += 1  # Increment win count for strategy2 against strategy1

                        pbar.update(1)


                        # print(f"{k} Player 1 : {strategy1}, Player 2 : {strategy2} result : {winner}")
                        print(f"Match {k} {strategy1} VS {strategy2} : result : {winner}")
                        
                        k+=1

    # if verbose:
    restore_print()


    # Step 6: Convert win counts in the grid to percentages
    # for i in range(num_strategies):
    #     for j in range(num_strategies):
    #         if i != j:  # Skip self-matches
    #             total_games = rounds  # Number of rounds played between strategy i and strategy j
    #             win_grid[i, j] = (win_grid[i, j] / total_games) * 100  # Convert win counts to percentages

    # Step 7: Return the results and the win percentage grid
    return results, win_grid

    # return results

In [232]:
def single_match(strategy1, strategy2):
    agent1 = Agent(player_number=1, strategy=strategy1)
    agent2 = Agent(player_number=2, strategy=strategy2)
    winner = play_game(agent1=agent1, agent2=agent2)



In [233]:
single_match('minmax', 'greedy')


 | | 
-----
 | | 
-----
 | | 
-----
Available moves: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Bot (X) selects move: 0

X| | 
-----
 | | 
-----
 | | 
-----
Available moves: [1, 2, 3, 4, 5, 6, 7, 8]
Bot (O) selects move: 1

X|O| 
-----
 | | 
-----
 | | 
-----
Available moves: [2, 3, 4, 5, 6, 7, 8]
Bot (X) selects move: 3

X|O| 
-----
X| | 
-----
 | | 
-----
Available moves: [2, 4, 5, 6, 7, 8]
Bot (O) selects move: 8

X|O| 
-----
X| | 
-----
 | |O
-----
Available moves: [2, 4, 5, 6, 7]
Bot (X) selects move: 4

X|O| 
-----
X|X| 
-----
 | |O
-----
Available moves: [2, 5, 6, 7]
Bot (O) selects move: 6

X|O| 
-----
X|X| 
-----
O| |O
-----
Available moves: [2, 5, 7]
Bot (X) selects move: 5
X|O| 
-----
X|X|X
-----
O| |O
-----
Player 1 wins!


In [234]:
board = np.array([1,1,2,0,0,2,0,0,0])
print_board(board)

X|X|O
-----
 | |O
-----
 | | 
-----


In [236]:
# minimax(board, 5,)

In [237]:
single_match('random', 'minmax')


 | | 
-----
 | | 
-----
 | | 
-----
Available moves: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Bot (X) selects move: 3

 | | 
-----
X| | 
-----
 | | 
-----
Available moves: [0, 1, 2, 4, 5, 6, 7, 8]
Bot (O) selects move: 0

O| | 
-----
X| | 
-----
 | | 
-----
Available moves: [1, 2, 4, 5, 6, 7, 8]
Bot (X) selects move: 2

O| |X
-----
X| | 
-----
 | | 
-----
Available moves: [1, 4, 5, 6, 7, 8]
Bot (O) selects move: 4

O| |X
-----
X|O| 
-----
 | | 
-----
Available moves: [1, 5, 6, 7, 8]
Bot (X) selects move: 5

O| |X
-----
X|O|X
-----
 | | 
-----
Available moves: [1, 6, 7, 8]
Bot (O) selects move: 8
O| |X
-----
X|O|X
-----
 | |O
-----
Player 2 wins!


In [238]:


# Example usage
# strategies = ['random', 'greedy', 'block', 'center_corner']
strategies = ['random', 'greedy', 'block', 'center_corner' ,'greedy_center', 'minmax']
# strategies = ['random', 'greedy', 'block', 'center', 'corner', 'edge', 'alpha-beta']
rounds = 1
VERBOSE = False
SELF_GAMES = False
tournament_results, win_grid = round_robin_tournament(strategies, rounds, verbose= VERBOSE, self_games = SELF_GAMES)

# Display results
print()

for strategy, result in tournament_results.items():
    print(f"{strategy}: Wins: {result['wins']}, Losses: {result['losses']}, Draws: {result['draws']}")
    


Playing matches:  50%|█████     | 15/30 [00:00<00:00, 149.53it/s]

Playing matches: 100%|██████████| 30/30 [00:00<00:00, 35.50it/s] 


random: Wins: 0, Losses: 7, Draws: 3
greedy: Wins: 3, Losses: 5, Draws: 2
block: Wins: 3, Losses: 5, Draws: 2
center_corner: Wins: 4, Losses: 6, Draws: 0
greedy_center: Wins: 5, Losses: 0, Draws: 5
minmax: Wins: 8, Losses: 0, Draws: 2


## Display results

In [239]:
tournament_results

{'random': {'wins': 0, 'losses': 7, 'draws': 3},
 'greedy': {'wins': 3, 'losses': 5, 'draws': 2},
 'block': {'wins': 3, 'losses': 5, 'draws': 2},
 'center_corner': {'wins': 4, 'losses': 6, 'draws': 0},
 'greedy_center': {'wins': 5, 'losses': 0, 'draws': 5},
 'minmax': {'wins': 8, 'losses': 0, 'draws': 2}}

In [240]:
win_grid

array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 0.],
       [1., 1., 0., 1., 0., 0.],
       [2., 1., 1., 0., 0., 0.],
       [1., 1., 1., 2., 0., 0.],
       [2., 2., 2., 2., 0., 0.]])

In [241]:
from tabulate import tabulate

# Assume strategies is a list of strategy names
# Assume tournament_results is a dictionary holding results for each strategy
total_matches = 2 * len(strategies) * (len(strategies) - 1) * rounds
matches_per_strat = total_matches/len(strategies)
# Organize results into a list of lists
results = [
    [strategy, info['wins'], info['losses'], info['draws'],  # Absolute counts
     f"{(info['wins'] / matches_per_strat) * 100:.2f}%",  # Win %
     f"{(info['losses'] / matches_per_strat) * 100:.2f}%",  # Loss %
     f"{(info['draws'] / matches_per_strat) * 100:.2f}%"]  # Draw %
    for strategy, info in tournament_results.items()
]

# Create a table string
table_string = tabulate(results, headers=['Strategy', 'Wins', 'Losses', 'Draws', 'Win %', 'Loss %', 'Draw %'], tablefmt='grid')

# Print the table
print(table_string)

# Optionally save the table to a .txt file
# with open('tournament_results.txt', 'w') as file:
#     file.write(table_string)


+---------------+--------+----------+---------+---------+----------+----------+
| Strategy      |   Wins |   Losses |   Draws | Win %   | Loss %   | Draw %   |
+===============+========+==========+=========+=========+==========+==========+
| random        |      0 |        7 |       3 | 0.00%   | 70.00%   | 30.00%   |
+---------------+--------+----------+---------+---------+----------+----------+
| greedy        |      3 |        5 |       2 | 30.00%  | 50.00%   | 20.00%   |
+---------------+--------+----------+---------+---------+----------+----------+
| block         |      3 |        5 |       2 | 30.00%  | 50.00%   | 20.00%   |
+---------------+--------+----------+---------+---------+----------+----------+
| center_corner |      4 |        6 |       0 | 40.00%  | 60.00%   | 0.00%    |
+---------------+--------+----------+---------+---------+----------+----------+
| greedy_center |      5 |        0 |       5 | 50.00%  | 0.00%    | 50.00%   |
+---------------+--------+----------+---

In [242]:
# Assume win_grid is the MxM numpy array holding win percentages between strategies
# win_grid = np.array([[0, 33.33, 66.67], [66.67, 0, 33.33], [33.33, 66.67, 0]])  # Example data

win_grid = win_grid
# Create a list of lists with strategies as rows and columns, including the win percentages
grid_results = [
    ([strategies[i]] + [f"{win_grid[i, j] * 100 / 2:.2f}%" for j in range(len(strategies))])
    for i in range(len(strategies))
]

# Add the strategy names as headers (the first row/column)
headers = ["Strategy"] + strategies

# Create the table string using tabulate
table_string = tabulate(grid_results, headers=headers, tablefmt='grid')

# Print the table
print(table_string)

+---------------+----------+----------+---------+-----------------+-----------------+----------+
| Strategy      | random   | greedy   | block   | center_corner   | greedy_center   | minmax   |
+===============+==========+==========+=========+=================+=================+==========+
| random        | 0.00%    | 0.00%    | 0.00%   | 0.00%           | 0.00%           | 0.00%    |
+---------------+----------+----------+---------+-----------------+-----------------+----------+
| greedy        | 50.00%   | 0.00%    | 50.00%  | 50.00%          | 0.00%           | 0.00%    |
+---------------+----------+----------+---------+-----------------+-----------------+----------+
| block         | 50.00%   | 50.00%   | 0.00%   | 50.00%          | 0.00%           | 0.00%    |
+---------------+----------+----------+---------+-----------------+-----------------+----------+
| center_corner | 100.00%  | 50.00%   | 50.00%  | 0.00%           | 0.00%           | 0.00%    |
+---------------+----------+--

In [243]:
grid_results

[['random', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%'],
 ['greedy', '50.00%', '0.00%', '50.00%', '50.00%', '0.00%', '0.00%'],
 ['block', '50.00%', '50.00%', '0.00%', '50.00%', '0.00%', '0.00%'],
 ['center_corner', '100.00%', '50.00%', '50.00%', '0.00%', '0.00%', '0.00%'],
 ['greedy_center', '50.00%', '50.00%', '50.00%', '100.00%', '0.00%', '0.00%'],
 ['minmax', '100.00%', '100.00%', '100.00%', '100.00%', '0.00%', '0.00%']]